# 2. Query Expansion and Indexing

0. [Vector Space Retrieval](#Vector-Space-Retrieval)
1. [Local approach for extending queries: User Relevance Feedback (Rocchio Algorithm)](#Question-1---Relevance-Feedback)
2. [Distributed Information Retrieval (Fagin's Algorithm)](#Question-2----Distributed-Information-Retrieval)
3. [Inverted Files (Hard)](#Question-3---Inverted-Files)

In [21]:
class_documents = ['B1 A course on Integral equations',
    'B2 Attractors for semigroups and evolution equations',
    'B3 Automatic differentiation of algorithms: theory, implementation, and application',
    'B4 Geometrical aspects of partial differential equations',
    'B5 Ideals, varieties and algorithms: an introduction to computational algebraic geometry and commutative algebra',
    'B6 Introduction to Hamiltonian dynamical systems and the N-body problem',
    'B7 Knapsack problems: Algorithms and computer implementations',
    'B8 Methods of solving singular systems of ordinary differential equations with delay',
    'B9 Nonlinear systems',
    'B10 Ordinary differential equations',
    'B11 Oscillation theory for neutral differential equations with delay',
    'B12 Oscillation theory for delay differential equations',
    'B13 Pseudodifferential operators and nonlinear partial differential equations',
    'B14 Sinc methods for quadrature and differential equations',
    'B15 Stability of stochastic differential equations with respect to semi-martingales',
    'B16 The boundary integral approach to static and dynamic contact problems',
    'B17 The double mellin-barnes type integrals and their applications to convolution theory']

## Vector Space Retrieval
The following code is modified from the previous lab. It is used to construct the vocabulary and vectorize the documents and query.

In [2]:
# Loading of libraries and documents
import string
import math

import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from collections import Counter
import numpy as np

from operator import itemgetter

stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /home/lucia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Tokenize, stem a document
def tokenize(text):
    """Given a string, removes the punctuation and returns
    array of stemmed words.
    """
    # Remove punctuation
    text = "".join([ch for ch in text if ch
        not in string.punctuation])
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Return stemmed document
    return " ".join([stemmer.stem(word.lower())
        for word in tokens])


def read_documents(filename):
    # Read documents from file (each line is a document)
    with open(filename) as f:
        content = f.readlines()
    original_documents = [x.strip() for x in content] 
    documents = [tokenize(d).split() for d in original_documents]
    return documents, original_documents


def create_vocabulary(documents):
    # Create the vocabulary
    # flatten 'documents'
    vocabulary = set([item for sublist in documents
        for item in sublist])
    # remove stopwords and sort
    vocabulary = [word for word in vocabulary if
        word not in stopwords.words('english')]
    vocabulary.sort()
    return vocabulary


# Compute IDF, storing values in a dictionary
def idf_values(vocabulary, documents):
    """Computes IDF as log(N/nt)
    where nt is the number of documents where the
    term t appears.
    """
    idf = {}
    num_documents = len(documents)
    for term in vocabulary:
        idf[term] = [term in x for x in documents]
        idf[term] = math.log(num_documents /
            Counter(idf[term])[True], math.e)
    return idf


# Generate the vector for a document (with normalization)
def vectorize(document, vocabulary, idf):
    """Returns TF-IDF as TF*IDF
    Computes TF as the term frequency, which means
    ftd/max_count
    where:
        ftd = nb. of times term t occurs in doc d
        max_count = nb. of times most common term occurs in d
    This is known as 'augmented frequency' and prevents
    a bias towards longer documents.
    """
    tfidf = [0] * len(vocabulary)
    tf = [0] * len(vocabulary)
    counts = Counter(document)
    # get the count for most common word
    max_count = counts.most_common(1)[0][1]
    for i, term in enumerate(vocabulary):
        tf[i] = counts[term] / max_count 
        tfidf[i] = tf[i] * idf[term]
    return tfidf


# Compute cosine similarity
def cosine_similarity(v1, v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]
        y = v2[i]
        sumxx += x * x
        sumyy += y * y
        sumxy += x * y
    if sumxy == 0:
        return 0
    else:
        return sumxy / math.sqrt(sumxx * sumyy)
    

def vectorize_query(query, vocabulary, idf):
    # get array of vectorized query words
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    # get the vector for the query
    query_vector = vectorize(q, vocabulary, idf)
    return query_vector
   
    
# Compute the search result (get topk documents)
def search_vec(query, topk):
    query_vector = vectorize_query(query, vocabulary, idf)
    # compute cosine similarity scores
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d]
        for d in range(len(documents))]
    # sort and return search result
    scores.sort(key=lambda x: -x[0])
    ans = []
    indices = []
    for i in range(topk):
        ans.append(original_documents[scores[i][1]])
        indices.append(scores[i][1])
    return ans, indices, query_vector

        
# Put everything together
def search_scores_vec(query, topk, vocabulary, documents,
    original_documents, idf=None, document_vectors=None):

    def search_vec_2(query, topk, vocabulary, idf, document_vectors,
        original_documents):
        query_vector = vectorize_query(query, vocabulary, idf)
        # compute cosine similarity scores
        scores = [[cosine_similarity(query_vector, document_vectors[d]), d]
            for d in range(len(documents))]
        # sort and return search result
        scores.sort(key=lambda x: -x[0])
        ans = []
        indices = []
        for i in range(topk):
            ans.append(original_documents[scores[i][1]])
            indices.append(scores[i][1])
        return ans, indices, query_vector
    
    if idf is None:
        idf = idf_values(vocabulary, documents)
    if document_vectors is None:
        document_vectors = [vectorize(d, vocabulary, idf) for d in documents]
    ans, indices, query_vector = search_vec_2(query, topk, vocabulary,
        idf, document_vectors, original_documents)
    for d in ans:
        print(d)
    return ans, indices, query_vector

## Question 1 - Relevance Feedback

In this exercise we will implement and test Rocchio's method for user relevance feedback.

Its objective is to find a query $\vec{q}_{opt}$ that optimally separates relevant ($D_r$) and non-relevant ($D_{nr}$) documents:

\begin{equation}
\vec{q}_{opt} = \text{arg max}_{\vec{q}}[\text{sim}(\vec{q}, \mu(D_r))-\text{sim}(\vec{q}, \mu(D_{nr}))]
\end{equation}

Let the set of relevant documents to be $D_r $ and the set of non-relevant documents to be $D_{nr}$. Then the modified query  $\vec{q_m}$  according to the Rocchio method is:

\begin{equation}
\vec{q_m} = \alpha \vec{q_0} + \frac{\beta}{|D_r|} \sum_{\vec{d_j} \in D_r} \vec{d_j} - \frac{\gamma}{|D_{nr}|} \sum_{\vec{d_k} \in D_{nr}} \vec{d_k}
\end{equation}

With:

| Variable    | Value                       |
|-------------|-----------------------------|
| $\vec{q_m}$ | Modified Query Vector       |
| $\vec{q_0}$ | Original Query Vector       |
| $\vec{d_j}$ | Related Document Vector     |
| $\vec{d_k}$ | Non-Related Document Vector |
| $\alpha$    | Original Query Weight       |
| $\beta$     | Related Documents Weight    |
| $\gamma$    | Non-Related Documents Weight|
| $D_r$       | Set of Related Documents    |
| $D_{nr}$    | Set of Non-Related Documents|

In the Rocchio algorithm negative term weights are ignored. This means that the negative term weights in $\vec{q_m}$ are set to 0.

First, complete the implementation of the Rocchio relevance feedback method, by adding the missing code for the function $expand\_query$.   

Then, compare the result obtained with the new query to the unmodified one.

In [4]:
def expand_query(relevant_doc_vecs, non_relevant_doc_vecs,
    query_vector, alpha, beta, gamma):
    
    relevant_doc_vecs = np.array(relevant_doc_vecs)
    non_relevant_doc_vecs = np.array(non_relevant_doc_vecs)
    query_vector = np.array(query_vector)
    'Dogs are predators and scavengers and the dog has powerful muscles'
    num_rel = len(relevant_doc_vecs)
    num_non_rel = len(non_relevant_doc_vecs)
    
    # Compute the first term in the Rocchio equation
    norm_query_vector = alpha * query_vector
    
    # Compute the second term in the Rocchio equation
    norm_sum_relevant = (beta / num_rel *
        relevant_doc_vecs.sum(axis=0))
    
    # Compute the last term in the Rocchio equation
    norm_sum_non_relevant = (gamma / num_non_rel *
        non_relevant_doc_vecs.sum(axis=0))
    
    # Sum all the terms
    modified_query_vector = (norm_query_vector +
        norm_sum_relevant - norm_sum_non_relevant)
    
    # Ignore negative elements
    modified_query_vector = np.maximum(modified_query_vector,
        np.zeros(len(query_vector)))

    return modified_query_vector.tolist()

The following code will gets the results for queries "baking" and "computer science", respectively:


In [23]:
original_documents = class_documents
documents = [tokenize(d).split() for d in original_documents]
# Create vocabulary
vocabulary = create_vocabulary(documents)

# Compute IDF values and vectors
topk = 4
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(d, vocabulary, idf) for d in documents]
ans, result_doc_ids, query_vector = search_scores_vec("application theory",
    topk, vocabulary, documents, original_documents,
    idf=idf, document_vectors=document_vectors)

B3 Automatic differentiation of algorithms: theory, implementation, and application
B17 The double mellin-barnes type integrals and their applications to convolution theory
B12 Oscillation theory for delay differential equations
B11 Oscillation theory for neutral differential equations with delay


The following code produces the result for the unmodified query:

In [24]:
def get_unmodified_query(relevant_indices, ans, result_doc_ids, query_vector,
    alpha, beta, gamma, document_vectors, vocabulary):
    
    def items_from_list(indices, alist):
        items = itemgetter(*indices)(alist)
        if type(items) == tuple:
            return list(items)
        else:
            return [items]
        
    # remove relevant indices from all indices
    non_relevant_indices =  [n for n in range(len(ans)) if n not in relevant_indices]
    # get indices of relevant and non-relevant results
    relevant_doc_ids = items_from_list(relevant_indices, result_doc_ids)
    non_relevant_doc_ids = items_from_list(non_relevant_indices, result_doc_ids)
    # get vectors of relevant and non-relevant results
    relevant_doc_vecs = items_from_list(relevant_doc_ids, document_vectors)
    non_relevant_doc_vecs = items_from_list(non_relevant_doc_ids, document_vectors)
    # expand query
    expanded_query = expand_query(relevant_doc_vecs,
        non_relevant_doc_vecs, query_vector, alpha, beta, gamma)
    # obtain new query vocabulary
    new_query = ' '.join([vocabulary[i] for i, val in
        enumerate(expanded_query) if val > 0])
    
    return new_query

Examples:

In [26]:
# list of indices marked as relevant
relevant_indices = [0]

# obtain new query vocabulary
new_query = get_unmodified_query(relevant_indices,
    ans, result_doc_ids, query_vector, 1, .75, .25,
    document_vectors, vocabulary)

# obtain answer for new query
new_ans, new_result_doc_ids, new_query_vector = search_vec(new_query, 4)

print('Modified query: ', new_query)
new_ans

Modified query:  algorithm applic automat b3 differenti implement theori


['B3 Automatic differentiation of algorithms: theory, implementation, and application',
 'B7 Knapsack problems: Algorithms and computer implementations',
 'B17 The double mellin-barnes type integrals and their applications to convolution theory',
 'B12 Oscillation theory for delay differential equations']

## Question 2 -  Distributed Information Retrieval

In this exercise we implement a core process of Fagin's algorithm, the parallel scanning of the posting lists. Assume an aggregation function that returns the sum of the TF-IDF scores given the terms in a document.

We assume that the posting lists for each term of a retrieval system are running on a different node.

We first create a dictionary $h$, where each entry of the dictionary corresponds to a posting list for term, assumed to be hosted in a different node. Explore the structure of $h$, to understand it. We suggest to first use the simple collection breads.txt.

In [5]:
def create_dictionary_h(document_vectors, vocabulary, original_documents):
    doc_vecs = np.array(document_vectors)
    h = {}
    for i, term in enumerate(vocabulary):
        ht = {}
        for docj in range(len(original_documents)):
            tfidf = doc_vecs[docj][i]
            ht[docj] = tfidf
        # sort by higher tf-idf
        sorted_ht = sorted(ht.items(), key=itemgetter(1), reverse=True)
        h[term] = sorted_ht
    return h

### Complete the following function that implements the Fagin algorithm.

In [6]:
def fagin_algorithm(query, h, k, vocabulary): 
    def prepare_vbles(query, h):
        # Split and stem the query
        q = query.split()
        q = set([stemmer.stem(w) for w in q])
        query_term_cnt = len(q)
    
        # Select the posting lists for the query terms
        posting_lists = {}
        for term in q:
            if term in h:
                posting_lists[term] = h[term]
                
        # Count documents
        max_len = len(h[list(h.keys())[0]])

        return q, query_term_cnt, posting_lists, max_len
    
    def phase_1_Fagin(posting_lists, k, query_term_cnt):
        """Phase 1 of Fagin's algorithm
        Traverses the selected posting lists until finding k documents
        that appear in all posting lists, then stops.

        The result is a dictionary documents_occurrences, with:
            document identifiers as keys, 
            number of documents found as value.
        """        
        x = list(posting_lists.values())
        documents_occurrences = {}
        for j in range(max_len):
            for i in x:
                (idx, value) = i[j]
                if idx in documents_occurrences:
                    documents_occurrences[idx] = documents_occurrences[idx] + 1
                else:
                    documents_occurrences[idx] = 1
                    
                if list(documents_occurrences.values()).count(query_term_cnt) >= k:
                    break
            if list(documents_occurrences.values()).count(query_term_cnt) >= k:
                break
        print(documents_occurrences)
        return documents_occurrences


    def phase_2_Fagin(posting_lists, documents_occurrences, q):
        """Retrieve for the found documents the tfidf values and add them up.
        For simplicity, do not distinguish the cases when
            the values have already been retrieved in the previous phase, or
            we use random access to obtain those values
        """
        tfidf = {}
        for d in documents_occurrences.keys():
            t = 0
            for term in q:
                t = t + dict(posting_lists[term])[d]
            tfidf[d] = t
        return tfidf
        
    q, query_term_cnt, posting_lists, max_len = prepare_vbles(query, h)
    documents_occurrences = phase_1_Fagin(posting_lists, k, query_term_cnt)
    tfidf = phase_2_Fagin(posting_lists, documents_occurrences, q)

    # Lastly, compute the top-k documents and return the answer
    ans = sorted(tfidf.items(), key=lambda x:x[1], reverse = True)[:k]
    return ans

In [124]:
ans = fagin_algorithm("bread recipe", h, 2, vocabulary)
print(ans)
for document_id in ans:
    print(original_documents[document_id[0]])

# Provided solution:
#{3: 2, 2: 1, 0: 2, 1: 1}
#[(3, 1.1394342831883648), (0, 0.56971714159418241)]
#Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
#How to Bake Breads Without Baking Recipes

{3: 2, 2: 1, 0: 2, 1: 1}
[(3, 1.1394342831883648), (0, 0.56971714159418241)]
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
How to Bake Breads Without Baking Recipes


Produce your own datasets to explore the behavior of the algorithm. Create a dataset such that for retrieving a 2 term query a total of 6 documents needs to be retrieved in the first phase of the algorithm (as in the example in the lecture).

In [15]:
original_documents = ['Dog breeds are obtained by mating selected dogs to maintain dog characteristics',
    'Dogs are predators and scavengers and the dog has powerful muscles',
    'My cousin was bitten by a dog and now hates dogs',
    'Dog breeds are lame but cat breeds and hedgehog breeds are even worse',
    'Breeding animals of one breed with animals of another breed are mixed breeds',
    'Plant breeds are obtained by breeding plants']

documents = [tokenize(d).split() for d in original_documents]
# Create vocabulary
vocabulary = create_vocabulary(documents)

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(d, vocabulary, idf) for d in documents]

h = create_dictionary_h(document_vectors, vocabulary, original_documents)

ans = fagin_algorithm("dog breeding", h, 2, vocabulary)
print(ans)
for document_id in ans:
    print(original_documents[document_id[0]])

{0: 2, 3: 2, 1: 1, 4: 1, 2: 1, 5: 1}
[(0, 0.54062014414421911), (3, 0.54062014414421911)]
Dog breeds are obtained by mating selected dogs to maintain dog characteristics
Dog breeds are lame but cat breeds and hedgehog breeds are even worse


## Question 3 - Inverted Files

An inverted list $l_k$ contains the number of documents in which term $k$ occurs ($f_k$) and the list of document identifiers containing $k$ ($d_{i1}$ ... $d_{if_k}$)

Inverted files are built by concatenating the inverted lists for all terms occurring in the document collection:

Now assume an inverted list that stores not only the document identifiers of the documents in which the term appears, but also an *offset* of the appearance of a term in a document.

An *offset* of a term $l_k$ given a document is defined as the number of words between the start of the document and $l_k$. Thus our inverted list is now:

$l_k= \langle f_k: \{d_{i_1} \rightarrow [o_1,\ldots,o_{n_{i_1}}]\}, 
\{d_{i_2} \rightarrow [o_1,\ldots,o_{n_{i_2}}]\}, \ldots, 
\{d_{i_k} \rightarrow [o_1,\ldots,o_{n_{i_k}}]\} \rangle$



This means that in document $d_{i_1}$ term $k$ appears $n_{i_1}$ times and at offset $[o_1,,o_{n_{i_1}}]$, where $[o_1,,o_{n_{i_1}}]$ are sorted in ascending order, these type of indices are also known as term-offset indices. An example of a term-offset index is as follows:

**Obama** = $⟨4 : {1 → [3]},{2 → [6]},{3 → [2,17]},{4 → [1]}⟩$

**Governor** = $⟨2 : {4 → [3]}, {7 → [14]}⟩$

**Election** = $⟨4 : {1 → [1]},{2 → [1,21]},{3 → [3]},{5 → [16,22,51]}⟩$

Which is to say that the term **Governor** appear in 2 documents. In document 4 at offset 3, in document 7 at offset 14.

Now let us consider the _**SLOP/x**_ operator in text retrieval, which has the syntax: *QueryTerm1 SLOP/x QueryTerm2* (with $x$ an integer positive number $x \geq 1$).

It finds occurrences of *QueryTerm1* within $x$ (but not necessarily in that order) words of *QueryTerm2*. Thus $x = 1$ demands that *QueryTerm1* be adjacent to *QueryTerm2*.

In [42]:
lk = {"obama":(4, {1:[3], 2:[6], 3:[2,17], 4:[1]}),
      "governor":(2, {4:[3], 7:[14]}),
      "election":(4, {1:[1], 2:[1,21], 3:[3], 5:[16,22,51]})}

In [44]:
def slop_x(query_term_1, query_term_2, x):
    docs_found = []
    for d in lk[query_term_1][1]:
        try:
            q2 = lk[query_term_2][1][d]
            for o1 in lk[query_term_1][1][d]:
                for o2 in q2:
                    if d in docs_found:
                        break
                    if abs(o1 - o2) == x:
                        docs_found.append(d)
                        break
        except KeyError:
            pass      

    return docs_found



1 List the set of documents that satisfy the query **Obama _SLOP/2_ Election**.

In [45]:
x = 2
query_term_1 = "obama"
query_term_2 = "election"
slop_x(query_term_1, query_term_2, x)

[1]

2 List each set of values for which the query **Obama _SLOP/x_ Election** has a different set of documents as answers (starting from $x = 1$). 

In [51]:
query_term_1 = "obama"
query_term_2 = "election"
answers = {}
for x in range(52):
    ans = slop_x(query_term_1, query_term_2, x)
    if len(ans) > 0:
        answers[x] = ans

answers

{1: [3], 2: [1], 5: [2], 14: [3], 15: [2]}



3 Consider the general procedure for "merging" two term-offset inverted lists for a given document, to determine where the document satisfies a *SLOP/x* clause (since in general there will be many offsets at which each term occurs in a document). Let $L$ denote the total number of occurrences of the two terms in the document. Assume we have a pointer to the list of occurrences of each term and can move the pointer along this list. As we do so we check whether we have a hit for *SLOP/x* (i.e. the *SLOP/x* clause is satisfied). Each move of either pointer counts as a step. Based on this assumption is there a general "merging" procedure to determine whether the document satisfies a *SLOP/x* clause, for which the following is true? Justify your answer.

1. The merge can be accomplished in a number of steps linear in $L$ regardless of $x$, and we can ensure that each pointer moves only to the right (i.e. forward).
2. The merge can be accomplished in a number of steps linear in $L$, but a pointer may be forced to move to the left (i.e. backwards).
3. The merge can require $x \times L$ steps in some cases.